# 8.5. Implementation of Recurrent Neural Networks from Scratch

In this section we will implement an RNN from scratch for a character-level language model, according to our descriptions in Section 8.4. Such a model will be trained on H. G. Wells’ The Time Machine. As before, we start by reading the dataset first, which is introduced in Section 8.3.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
use d2l::Vocab;
use d2l::SeqDataLoader;
IPerl->load_plugin('Chart::Plotly'); # Jupyter
#import Chart::Plotly 'show_plot'; # localmente

In [2]:
my ($batch_size, $num_steps) = (32, 35);
my ($train_iter, $vocab) = d2l->load_data_time_machine(batch_size      => $batch_size, 
                                                       num_steps       => $num_steps,
                                                       use_random_iter => 1);

CODE(0xc159c90)Vocab=HASH(0xc12dc50)

In [3]:
print "vocab:", dump $vocab;

vocab:bless({
  _token_freqs => [
                    [" ", 32926],
                    ["e", 19781],
                    ["t", 15155],
                    ["a", 12752],
                    ["i", 11312],
                    ["o", 11157],
                    ["n", 11012],
                    ["s", 9313],
                    ["r", 8895],
                    ["h", 8841],
                    ["d", 6868],
                    ["l", 6658],
                    ["m", 4420],
                    ["u", 4328],
                    ["c", 4049],
                    ["f", 3749],
                    ["w", 3524],
                    ["g", 3512],
                    ["y", 3024],
                    ["p", 2867],
                    ["b", 2171],
                    ["v", 1435],
                    ["k", 1231],
                    ["x", 282],
                    ["j", 187],
                    ["z", 108],
                    ["q", 106],
                  ],
  idx_to_token => [
                    "<unk>",
  

1

In [4]:
# Iterator
while ( my $minibatch = $train_iter->() ) {
  print "\n\nX:", dump($minibatch->{data}->asarray), "\n";
  print "\n\nY:", dump($minibatch->{label}->asarray), "\n\n";
  last;
}



X:[
  [
    1,
    7,
    6,
    17,
    1,
    10,
    2,
    1,
    8,
    4,
    5,
    11,
    1,
    4,
    16,
    3,
    2,
    9,
    1,
    8,
    6,
    13,
    2,
    3,
    5,
    13,
    2,
    1,
    21,
    9,
    5,
    18,
    10,
    3,
    2,
  ],
  [
    7,
    8,
    1,
    3,
    10,
    4,
    3,
    1,
    6,
    14,
    9,
    15,
    6,
    7,
    8,
    15,
    5,
    6,
    14,
    8,
    7,
    2,
    8,
    8,
    1,
    13,
    6,
    22,
    2,
    8,
    1,
    5,
    7,
    3,
    2,
  ],
  [
    1,
    12,
    5,
    7,
    2,
    1,
    6,
    16,
    1,
    3,
    10,
    5,
    15,
    23,
    7,
    2,
    8,
    8,
    1,
    7,
    5,
    12,
    1,
    10,
    4,
    8,
    1,
    7,
    6,
    1,
    9,
    2,
    4,
    12,
    2,
  ],
  [
    6,
    1,
    10,
    5,
    18,
    10,
    1,
    19,
    2,
    8,
    3,
    2,
    9,
    11,
    4,
    19,
    1,
    7,
    5,
    18,
    10,
    3,
    1,
    5,
    3,
    1,
    16,
    2,

## 8.5.1. One-Hot Encoding

Recall that each token is represented as a numerical index in `train_iter`.
Feeding these indices directly to a neural network might make it hard to
learn.
We often represent each token as a more expressive feature vector.
The easiest representation is called *one-hot encoding*,
which is introduced
in Section 3.4.1.

In a nutshell, we map each index to a different unit vector: assume that the number of different tokens in the vocabulary is $N$ (`len(vocab)`) and the token indices range from 0 to $N-1$.
If the index of a token is the integer $i$, then we create a vector of all 0s with a length of $N$ and set the element at position $i$ to 1.
This vector is the one-hot vector of the original token. The one-hot vectors with indices 0 and 2 are shown below.


In [5]:
print $vocab->len;
print "\nTokens to ids: ", dump $vocab->getitem([ " ", "b", "u", "t", " ", "s", "o", "m", "e", " " ]);
print "\nIds to tokens: ", dump $vocab->to_tokens([ 1, 21, 14, 3, 1, 8, 6, 13, 2, 1 ]);

28
Tokens to ids: [1, 21, 14, 3, 1, 8, 6, 13, 2, 1]
Ids to tokens: [" ", "b", "u", "t", " ", "s", "o", "m", "e", " "]

1

In [6]:
print dump mx->nd->one_hot(mx->nd->array([0, 2]), $vocab->len)->asarray;

[
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]

1

The shape of the minibatch that we sample each time is (batch size, number of time steps).
The `one_hot` function transforms such a minibatch into a three-dimensional tensor with the last dimension equals to the vocabulary size (`len(vocab)`).
We often transpose the input so that we will obtain an
output of shape
(number of time steps, batch size, vocabulary size).
This will allow us
to more conveniently
loop through the outermost dimension
for updating hidden states of a minibatch,
time step by time step.


In [7]:
my $X = mx->nd->arange(stop => 10)->reshape([2, 5]);
print dump mx->nd->one_hot($X->transpose, 28)->shape;

[5, 2, 28]

1

## 8.5.1. Initializing the Model Parameters

Next, we initialize the model parameters for
the RNN model.
The number of hidden units `num_hiddens` is a tunable hyperparameter.
When training language models,
the inputs and outputs are from the same vocabulary.
Hence, they have the same dimension,
which is equal to the vocabulary size.

In [ ]:
sub get_params{ 
    my ($vocab_size, $num_hiddens, $device) = @_;
    my $num_outputs = $vocab_size;
    my $num_inputs  = $num_outputs;
    sub normal {
        my ($shape, $device) = @_;
        return mx->nd-normal(scale => 0.01, size => $shape, ctx => $device);
    }
    # Hidden layer parameters
    my $W_xh = normal([$num_inputs, $num_hiddens], $device);
    my $W_hh = normal([$num_inputs, $num_hiddens], $device);
    my $b_h = mx->nd->zeros($num_hiddens, ctx => $device);
    # Output layer parameters
    my $W_hq = normal([$num_hiddens, $num_outputs], $device);
    my $b_q = mx->nd->zeros($num_outputs, ctx => $device);
    # Attach gradients
    my $params = [$W_xh, $W_hh, $b_h, $W_hq, $b_q];
    foreach my $param (@$params) {
        $param->attach_grad();
    }
    return $params;
}

In [ ]:
print dump get_params($vocab->len, 512, d2l->try_gpu());

## 8.5.3. RNN Model

To define an RNN model,
we first need an `init_rnn_state` function
to return the hidden state at initialization.
It returns a tensor filled with 0 and with a shape of (batch size, number of hidden units).
Using tuples makes it easier to handle situations where the hidden state contains multiple variables,
which we will encounter in later sections.

In [ ]:
sub init_rnn_state { 
    my ($batch_size, $num_hiddens, $device) = @_;
    return (mx->nd->zeros([$batch_size, $num_hiddens], ctx => $device),);
}

The following `rnn` function defines how to compute the hidden state and output
at a time step.
Note that
the RNN model
loops through the outermost dimension of `inputs`
so that it updates hidden states `H` of a minibatch,
time step by time step.
Besides,
the activation function here uses the $\tanh$ function.
As
described in :numref:`sec_mlp`, the
mean value of the $\tanh$ function is 0, when the elements are uniformly
distributed over the real numbers.

In [ ]:
sub rnn{ 
    my ($inputs, $state, $params) = @_;
    # Shape of `inputs`: (`num_steps`, `batch_size`, `vocab_size`)
    my ($W_xh, $W_hh, $b_h, $W_hq, $b_q) = $params;
    my $H = $state;
    my $outputs = [];
    # Shape of `X`: (`batch_size`, `vocab_size`)
    foreach $X (@$inputs){
        $H = mx->nd->tanh(mx->nd->dot($X, $W_xh) + mx->nd->dot($H, $W_hh) + $b_h);
        my $Y = mx->nd->dot($H, $W_hq) + $b_q;
        outputs.append($Y);
    }
    return (mx->nd->concatenate($outputs, axis => 0), ($H));
}

With all the needed functions being defined,
next we create a class to wrap these functions and store parameters for an RNN model implemented from scratch.

In [ ]:
package RNNModelScratch{  #@save
    #"""An RNN Model implemented from scratch."""
    sub new{
        my ($class, $vocab_size, $num_hiddens, $device, $get_params, $init_state, $forward_fn) = @_;
        my $self = {
            vocab_size => $vocab_size,
            num_hiddens =>  $num_hiddens,
            params => get_params($vocab_size, $num_hiddens, $device),
            init_state => $init_state,
            forward_fn => $forward_fn
        }
        bless $self, $class;
        return $self;
    }

    sub call{ 
        my $self = shift;
        my ($X, $state) = @_;
        $X = mx-nd->one_hot($X->transpose, $self->{vocab_size});
        return $self->{forward_fn}($X, $state, $self-{params});
    }
    
    sub begin_state{
        my $self = shift;
        my ($batch_size, $ctx) = @_;
        return $self->{init_state}($batch_size, self->{num_hiddens}, $ctx);
    }
}

Let us check whether the outputs have the correct shapes, e.g., to ensure that the dimensionality of the hidden state remains unchanged.

In [ ]:
my $num_hiddens = 512;
my $net = RNNModelScratch($vocab->len, $num_hiddens, d2l->try_gpu(), \&get_params, \&init_rnn_state, \&rnn);
my $state = $net->begin_state($X->shape[0], d2l->try_gpu());
my ($Y, $new_state)= $net->call($X->as_in_context(d2l->try_gpu()), $state);
print dump ($Y->shape, $new_state->len, $new_state->[0]->shape);

We can see that the output shape is (number of time steps $\times$ batch size, vocabulary size), while the hidden state shape remains the same, i.e., (batch size, number of hidden units).


## 8.5.4. Prediction

Let us first define the prediction function
to generate new characters following
the user-provided `prefix`,
which is a string containing several characters.
When looping through these beginning characters in `prefix`,
we keep passing the hidden state
to the next time step without
generating any output.
This is called the *warm-up* period,
during which the model updates itself
(e.g., update the hidden state)
but does not make predictions.
After the warm-up period,
the hidden state is generally better than
its initialized value at the beginning.
So we generate the predicted characters and emit them.


In [ ]:
syb predict_ch8 {#@save
    #Generate new characters following the `prefix`
    my ($prefix, $num_preds, $net, $vocab, $device) = @_;  
    my $state = net->begin_state(batch_size => 1, ctx => $device);
}

Now we can test the `predict_ch8` function.
We specify the prefix as `time traveller ` and have it generate 10 additional characters.
Given that we have not trained the network,
it will generate nonsensical predictions.

## 8.5.5. Gradient Clipping

For a sequence of length $T$,
we compute the gradients over these $T$ time steps in an iteration, which results in a chain of matrix-products with length  $\mathcal{O}(T)$ during backpropagation.
As mentioned in :numref:`sec_numerical_stability`, it might result in numerical instability, e.g., the gradients may either explode or vanish, when $T$ is large. Therefore, RNN models often need extra help to stabilize the training.

Generally speaking,
when solving an optimization problem,
we take update steps for the model parameter,
say in the vector form
$\mathbf{x}$,
in the direction of the negative gradient $\mathbf{g}$ on a minibatch.
For example,
with $\eta > 0$ as the learning rate,
in one iteration we update
$\mathbf{x}$
as $\mathbf{x} - \eta \mathbf{g}$.
Let us further assume that the objective function $f$
is well behaved, say, *Lipschitz continuous* with constant $L$.
That is to say,
for any $\mathbf{x}$ and $\mathbf{y}$ we have

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

In this case we can safely assume that if we update the parameter vector by $\eta \mathbf{g}$, then

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|,$$

which means that
we will not observe a change by more than $L \eta \|\mathbf{g}\|$. This is both a curse and a blessing.
On the curse side,
it limits the speed of making progress;
whereas on the blessing side,
it limits the extent to which things can go wrong if we move in the wrong direction.

Sometimes the gradients can be quite large and the optimization algorithm may fail to converge. We could address this by reducing the learning rate $\eta$. But what if we only *rarely* get large gradients? In this case such an approach may appear entirely unwarranted. One popular alternative is to clip the gradient $\mathbf{g}$ by projecting them back to a ball of a given radius, say $\theta$ via

$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$

By doing so we know that the gradient norm never exceeds $\theta$ and that the
updated gradient is entirely aligned with the original direction of $\mathbf{g}$.
It also has the desirable side-effect of limiting the influence any given
minibatch (and within it any given sample) can exert on the parameter vector. This
bestows a certain degree of robustness to the model. Gradient clipping provides
a quick fix to the gradient exploding. While it does not entirely solve the problem, it is one of the many techniques to alleviate it.

Below we define a function to clip the gradients of
a model that is implemented from scratch or a model constructed by the high-level APIs.
Also note that we compute the gradient norm over all the model parameters.


## 8.5.6 Training

Before training the model,
let us define a function to train the model in one epoch. It differs from how we train the model of :numref:`sec_softmax_scratch` in three places:

1. Different sampling methods for sequential data (random sampling and sequential partitioning) will result in differences in the initialization of hidden states.
1. We clip the gradients before updating the model parameters. This ensures that the model does not diverge even when gradients blow up at some point during the training process.
1. We use perplexity to evaluate the model. As discussed in :numref:`subsec_perplexity`, this ensures that sequences of different length are comparable.


Specifically,
when sequential partitioning is used, we initialize the hidden state only at the beginning of each epoch.
Since the $i^\mathrm{th}$ subsequence example  in the next minibatch is adjacent to the current $i^\mathrm{th}$ subsequence example,
the hidden state at the end of the current minibatch
will be
used to initialize
the hidden state at the beginning of the next minibatch.
In this way,
historical information of the sequence
stored in the hidden state
might flow over
adjacent subsequences within an epoch.
However, the computation of the hidden state
at any point depends on all the previous minibatches
in the same epoch,
which complicates the gradient computation.
To reduce computational cost,
we detach the gradient before processing any minibatch
so that the gradient computation of the hidden state
is always limited to
the time steps in one minibatch. 

When using the random sampling,
we need to re-initialize the hidden state for each iteration since each example is sampled with a random position.
Same as the `train_epoch_ch3` function in :numref:`sec_softmax_scratch`,
`updater` is a general function
to update the model parameters.
It can be either the `d2l.sgd` function implemented from scratch or the built-in optimization function in
a deep learning framework.


The training function supports an RNN model implemented either from scratch or using high-level APIs.



Now we can train the RNN model. Since we only use 10000 tokens in the dataset, the model needs more epochs to converge better.



Finally, let us check the results of using the random sampling method.

While implementing the above RNN model from scratch is instructive, it is not convenient. In the next section we will see how to improve the RNN model, such as how to make it easier to implement and make it run faster.

## 8.5.7. Summary
We can train an RNN-based character-level language model to generate text following the user-provided text prefix.

A simple RNN language model consists of input encoding, RNN modeling, and output generation.

RNN models need state initialization for training, though random sampling and sequential partitioning use different ways.

When using sequential partitioning, we need to detach the gradient to reduce computational cost.

A warm-up period allows a model to update itself (e.g., obtain a better hidden state than its initialized value) before making any prediction.

Gradient clipping prevents gradient explosion, but it cannot fix vanishing gradients.